In [50]:
# Importar librerías necesarias
from openai import OpenAI
import pandas as pd
import json
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import numpy as np
import random

# Crear una instancia del cliente de OpenAI
client = OpenAI()

In [51]:
# Dataset de pruebas
# Establecer semilla para reproducibilidad
random.seed(42)
np.random.seed(42)

# Crear una base de datos ficticia: "Estudio de comportamiento de aves en zonas urbanas y rurales"
n = 100

df_main = pd.DataFrame({
    "ID_observación": [f"OBS_{i+1:04d}" for i in range(n)],
    "Especie": np.random.choice(
        ["Zorzal", "Chincol", "Tenca", "Picaflor", "Gorrión", "Peuco", "Queltehue", "Diucón"], size=n),
    "Zona": np.random.choice(["Urbana", "Rural", "Suburbana"], size=n),
    "Hora_observación": np.random.choice(
        ["Mañana", "Mediodía", "Tarde", "Noche"], size=n),
    "Duración_observación_min": np.random.normal(loc=30, scale=12, size=n).round(1),
    "Cantidad_individuos": np.random.poisson(lam=4, size=n),
    "Temperatura_C": np.random.normal(loc=19, scale=4.5, size=n).round(1),
    "Humedad_%": np.random.normal(loc=55, scale=15, size=n).round(1),
    "Viento_kmh": np.random.normal(loc=12, scale=5, size=n).round(1),
    "Comportamiento": np.random.choice(
        ["Alimentación", "Canto", "Vuelo", "Reposo", "Interacción", "Defensa de territorio"], size=n),
    "Interacción_humana": np.random.choice(["Sí", "No"], size=n, p=[0.4, 0.6]),
    "Presencia_predador": np.random.choice(["Sí", "No"], size=n, p=[0.3, 0.7]),
    "Fecha_observación": pd.to_datetime(np.random.choice(pd.date_range("2023-01-01", "2023-12-31"), size=n)),
    "Observador": np.random.choice(
        ["Luis", "Camila", "Ignacio", "Valentina", "Marcelo", "Antonia"], size=n),
})

df_sample = df_main.head(3).to_string(index=False)

In [52]:
# Desplegar el DataFrame

df_main.head(10).style.set_table_styles([{
    'selector': 'th',  # Estilo para las celdas de la cabecera
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}, {
    'selector': 'td',  # Estilo para las celdas de datos
    'props': [('font-size', '10px')]  # Cambiar el tamaño de la fuente a pequeño
}])

,ID_observación,Especie,Zona,Hora_observación,Duración_observación_min,Cantidad_individuos,Temperatura_C,Humedad_%,Viento_kmh,Comportamiento,Interacción_humana,Presencia_predador,Fecha_observación,Observador
0,OBS_0001,Queltehue,Suburbana,Noche,30.300000,1,24.400000,35.000000,6.500000,Reposo,No,No,2023-06-18 00:00:00,Marcelo
1,OBS_0002,Picaflor,Rural,Noche,15.600000,5,22.000000,53.200000,0.100000,Interacción,No,Sí,2023-03-30 00:00:00,Luis
2,OBS_0003,Gorrión,Rural,Noche,39.500000,4,16.200000,89.200000,9.800000,Vuelo,Sí,Sí,2023-06-25 00:00:00,Camila
3,OBS_0004,Queltehue,Rural,Noche,35.200000,3,18.200000,13.200000,6.100000,Alimentación,No,No,2023-09-26 00:00:00,Marcelo
4,OBS_0005,Tenca,Rural,Noche,16.800000,4,17.500000,68.000000,25.000000,Defensa de territorio,No,Sí,2023-03-14 00:00:00,Camila
5,OBS_0006,Diucón,Rural,Mediodía,21.800000,3,16.900000,70.900000,8.700000,Vuelo,No,No,2023-09-02 00:00:00,Valentina
6,OBS_0007,Gorrión,Rural,Mediodía,40.700000,3,17.200000,34.600000,8.600000,Vuelo,Sí,No,2023-03-18 00:00:00,Valentina
7,OBS_0008,Gorrión,Suburbana,Tarde,37.300000,5,17.100000,48.000000,4.400000,Canto,No,Sí,2023-06-08 00:00:00,Valentina
8,OBS_0009,Queltehue,Suburbana,Noche,21.400000,3,16.900000,40.000000,10.700000,Canto,Sí,No,2023-11-20 00:00:00,Antonia
9,OBS_0010,Chincol,Rural,Mediodía,26.800000,2,23.300000,82.800000,8.700000,Canto,Sí,No,2023-04-08 00:00:00,Antonia


In [53]:
# Configuración GPT
gpt_model = 'gpt-4o' # gpt-3.5-turbo-0125 o gpt-4o

parametros = {
    "max_tokens": 1000,
    "temperature": 0.5,
    "top_p": 0.8,
    "presence_penalty": 0.3,
    "frequency_penalty": 0.5
}

In [54]:
# Crear el Contexto del Sistema
contexto_formato = (
    "Responde siempre en formato Markdown claro y estructurado.\n"
    "Utiliza código en bloques ```python``` cuando sea necesario, y explica cada paso antes o después del bloque de código.\n"
    "Usa visualizaciones con estilo `seaborn`, asegurándote de inicializar con `sns.set_theme()`.\n"
    "No muestres valores NaN a menos que se solicite específicamente.\n"
    "Accede al DataFrame completo con `df_main = globals()['df_main']`, "
    "aunque se te muestre solo un resumen (`df_sample`) como referencia visual.\n"
)

contexto_tema = (
    "Eres un asistente especializado en análisis de datos ecológicos. "
    "Analizas observaciones de aves en distintas condiciones (zona urbana/rural, temperatura, viento, interacción humana, etc.).\n"
    "Proporciona interpretaciones estadísticas y visuales que ayuden a comprender patrones de comportamiento y distribución de especies.\n"
)

contexto_sistema = contexto_formato + "\n" + contexto_tema

In [55]:
# Crear el Prompt e incorpora la Pregunta
def obtener_respuesta_con_dataframe(pregunta):
    prompt = (
    f"Se muestra una porción de un DataFrame con fines ilustrativos:\n\n"
    f"{df_sample}\n\n" 
    "Esta muestra es solo de referencia visual. El DataFrame completo está disponible como variable global `df_main`.\n\n"    
    f"Pregunta del usuario:\n{pregunta}\n\n"
    "Tu respuesta debe estar redactada en formato Markdown claro y ordenado. "
    "Incluye bloques de código Python cuando sea necesario usando la sintaxis ```python```.\n"
    "Recuerda seguir todas las instrucciones entregadas en el contexto del sistema, "
    "y asegúrate de usar `df_main` para cualquier operación de análisis real sobre los datos completos.\n"
)

    # Consulta API
    try:
        response = client.chat.completions.create(
            model=gpt_model,
            messages=[
                {"role": "system", "content": contexto_sistema},
                {"role": "user", "content": prompt}
            ],
            **parametros  # Usar el diccionario como argumento con **
        )
        
        respuesta = response.choices[0].message.content

        # Devolver la respuesta
        return respuesta
    except Exception as e:
        # Imprimir el error si ocurre algún problema
        print("Error al obtener respuesta de OpenAI:", str(e))

In [56]:
# Modulo repetible de consulta y respuesta
pregunta = (
    "1. ¿Cuál es la distribución de los comportamientos observados según la zona (urbana o rural)?\n"
    "2. ¿Qué especies son más comunes en zonas urbanas y cuáles en zonas rurales?\n"
    "3. ¿Existe una diferencia significativa en la duración promedio de observación según el tipo de comportamiento?\n"
    "4. ¿Cuáles son los rangos de temperatura más frecuentes en las observaciones con interacción humana?\n"
    "5. ¿Puedes generar un gráfico que muestre la relación entre la cantidad de individuos observados y la hora del día?"
)

respuesta = obtener_respuesta_con_dataframe(pregunta)

# Mostrar la respuesta utilizando Markdown
display(Markdown(respuesta))

# Análisis de Observaciones de Aves

A continuación, responderé las preguntas planteadas utilizando el DataFrame completo `df_main` que contiene observaciones de aves en distintas condiciones.

## 1. Distribución de Comportamientos según la Zona

Para analizar la distribución de los comportamientos observados según la zona (urbana o rural), utilizaremos un gráfico de barras.

```python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Acceder al DataFrame completo
df_main = globals()['df_main']

# Configurar el tema de seaborn
sns.set_theme()

# Crear un gráfico de barras para mostrar la distribución de comportamientos por zona
plt.figure(figsize=(10, 6))
sns.countplot(data=df_main, x='Comportamiento', hue='Zona')
plt.title('Distribución de Comportamientos según la Zona')
plt.xlabel('Comportamiento')
plt.ylabel('Frecuencia')
plt.legend(title='Zona')
plt.xticks(rotation=45)
plt.show()
```

Este gráfico nos permitirá visualizar qué comportamientos son más comunes en cada tipo de zona.

## 2. Especies Más Comunes en Zonas Urbanas y Rurales

Para determinar qué especies son más comunes en zonas urbanas y rurales, calcularemos las frecuencias para cada especie en ambas zonas.

```python
# Calcular las frecuencias de especies por zona
especies_urbanas = df_main[df_main['Zona'] == 'Urbana']['Especie'].value_counts()
especies_rurales = df_main[df_main['Zona'] == 'Rural']['Especie'].value_counts()

print("Especies más comunes en zonas urbanas:")
print(especies_urbanas)

print("\nEspecies más comunes en zonas rurales:")
print(especies_rurales)
```

Este análisis nos mostrará cuáles son las especies predominantes en cada tipo de zona.

## 3. Diferencia Significativa en la Duración Promedio Según el Comportamiento

Para evaluar si existe una diferencia significativa en la duración promedio de observación según el comportamiento, utilizaremos un análisis ANOVA.

```python
from scipy.stats import f_oneway

# Agrupar los datos por comportamiento y calcular la duración promedio
duracion_por_comportamiento = df_main.groupby('Comportamiento')['Duración_observación_min']

# Realizar ANOVA para comparar las medias entre grupos diferentes
anova_result = f_oneway(*(group for name, group in duracion_por_comportamiento))

print("Resultados del ANOVA:")
print(f"F-statistic: {anova_result.statistic}, p-value: {anova_result.pvalue}")
```

Un valor p menor a 0.05 indicaría una diferencia significativa entre al menos dos grupos.

## 4. Rangos de Temperatura Más Frecuentes con Interacción Humana

Para identificar los rangos de temperatura más frecuentes durante observaciones con interacción humana, crearemos un histograma.

```python
# Filtrar datos con interacción humana
interaccion_humana_df = df_main[df_main['Interacción_humana'] == 'Sí']

# Crear un histograma para mostrar los rangos de temperatura frecuentes
plt.figure(figsize=(10, 6))
sns.histplot(interaccion_humana_df['Temperatura_C'], bins=10)
plt.title('Rangos de Temperatura Más Frecuentes con Interacción Humana')
plt.xlabel('Temperatura (°C)')
plt.ylabel('Frecuencia')
plt.show()
```

El histograma mostrará cómo se distribuyen las temperaturas durante estas interacciones.

## 5. Relación entre Cantidad de Individuos Observados y Hora del Día

Finalmente, generaremos un gráfico que muestre esta relación mediante un scatter plot.

```python
# Crear un scatter plot para visualizar la relación entre cantidad y hora del día
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df_main, x='Hora_observación', y='Cantidad_individuos', hue='Zona', alpha=0.7)
plt.title('Relación entre Cantidad de Individuos Observados y Hora del Día')
plt.xlabel('Hora del Día')
plt.ylabel('Cantidad de Individuos')
plt.legend(title='Zona')
plt.xticks(rotation=45)
plt.show()
```

Este gráfico ayudará a visualizar si hay patrones específicos dependiendo del momento del día o la zona donde se realiza la observación.